In [3]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer, KNNImputer

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [4]:
#defining input at target 
x = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
y = heart['TenYearCHD']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [5]:
x_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3349.000000,3373.000000,3389.000000
mean,49.594395,0.495575,237.319797,25.769760,76.012688
std,8.589440,0.500054,45.272039,4.030293,12.156647
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.070000,68.000000
50%,49.000000,0.000000,234.000000,25.380000,75.000000
75%,56.000000,1.000000,265.000000,28.040000,83.000000
max,70.000000,1.000000,696.000000,51.280000,143.000000


In [6]:
x_test.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,839.000000,846.000000,848.000000
mean,49.547170,0.488208,234.333731,25.930579,75.344340
std,8.507661,0.500156,41.698751,4.273016,11.483547
min,33.000000,0.000000,135.000000,15.960000,46.000000
25%,43.000000,0.000000,205.000000,23.052500,67.000000
50%,48.500000,0.000000,232.000000,25.495000,75.000000
75%,56.000000,1.000000,259.000000,28.040000,82.000000
max,68.000000,1.000000,410.000000,56.800000,110.000000


In [7]:
# defining imputer
imputer = KNNImputer(n_neighbors = 5, weights = 'distance').fit(x_train)

# imputing missing values
x_train_imp = pd.DataFrame(imputer.transform(x_train), columns = x_train.columns)
x_test_imp = pd.DataFrame(imputer.transform(x_test), columns = x_test.columns)

In [8]:
x_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.000000,3390.000000,3390.000000
mean,49.594395,0.495575,237.303074,25.770697,76.012252
std,8.589440,0.500054,45.105039,4.025656,12.154880
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.080000,68.000000
50%,49.000000,0.000000,234.000000,25.380000,75.000000
75%,56.000000,1.000000,264.000000,28.040000,83.000000
max,70.000000,1.000000,696.000000,51.280000,143.000000


In [9]:
x_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.547170,0.488208,234.317956,25.933192,75.344340
std,8.507661,0.500156,41.604328,4.271077,11.483547
min,33.000000,0.000000,135.000000,15.960000,46.000000
25%,43.000000,0.000000,205.000000,23.057500,67.000000
50%,48.500000,0.000000,232.000000,25.495000,75.000000
75%,56.000000,1.000000,259.000000,28.040000,82.000000
max,68.000000,1.000000,410.000000,56.800000,110.000000


In [10]:
# Random Forest model
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(x_train_imp, y_train)

# predicting on test
rf_pred = rf_md.predict_proba(x_test_imp)[:,1]

# likilyhoods to labels
rf_label = np.where(rf_pred < 0.1, 0, 1)

print(classification_report(y_test, rf_label))

              precision    recall  f1-score   support

           0       0.94      0.40      0.56       719
           1       0.20      0.85      0.33       129

    accuracy                           0.47       848
   macro avg       0.57      0.63      0.44       848
weighted avg       0.83      0.47      0.52       848



In [12]:
# ada boost model
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 5), n_estimators = 500, learning_rate = .01).fit(x_train_imp, y_train)

# predicting on test
ada_pred = ada_md.predict_proba(x_test_imp)[:,1]

# likilyhoods to labels
ada_label = np.where(ada_pred < 0.1, 0, 1)

print(classification_report(y_test, ada_label))

              precision    recall  f1-score   support

           0       0.91      0.13      0.22       719
           1       0.16      0.93      0.27       129

    accuracy                           0.25       848
   macro avg       0.54      0.53      0.25       848
weighted avg       0.80      0.25      0.23       848



In [ ]:
# based on these results i would pick the random forest model to predict TenYearCHD because it has better overall performace